In [1]:
import pandas as pd
import numpy as np
import statistics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [2]:
qcm3 = pd.read_csv('Dataset/QCM3.csv', sep = ';')
qcm6 = pd.read_csv('Dataset/QCM6.csv', sep = ';')
qcm7 = pd.read_csv('Dataset/QCM7.csv', sep = ';')
qcm10 = pd.read_csv('Dataset/QCM10.csv', sep = ';')
qcm12 = pd.read_csv('Dataset/QCM12.csv', sep = ';')

In [3]:
print(qcm3.head())

   0.799_0.201  0.799_0.201.1  0.700_0.300  0.700_0.300.1  0.600_0.400  \
0       -10.06         -10.62       -14.43         -18.31       -24.64   
1        -9.69         -10.86       -16.73         -21.75       -28.47   
2       -12.07         -14.28       -21.54         -27.92       -35.19   
3       -14.21         -17.41       -25.91         -33.36       -41.29   
4       -16.57         -20.35       -29.97         -37.84       -47.03   

   0.600_0.400.1  0.501_0.499  0.501_0.499.1  0.400_0.600  0.400_0.600.1  \
0         -30.56       -38.62         -45.59       -54.89         -62.28   
1         -35.83       -43.65         -52.43       -61.92         -71.27   
2         -43.94       -52.04         -62.49       -71.97         -83.10   
3         -51.27       -59.94         -71.55       -81.51         -93.83   
4         -57.29       -67.13         -78.96       -90.01        -102.65   

   1-Octanol  1-Propanol  2-Butanol  2-propanol  1-isobutanol  
0          1           0          

In [4]:
dataset = pd.concat([qcm3, qcm6, qcm7, qcm10, qcm12])
print("Shape of dataset: ", dataset.shape)

Shape of dataset:  (125, 15)


In [5]:
"""dataset.loc[dataset["1-Octanol"] == 1, 'alcohol'] = '1-Octanol'
dataset.loc[dataset["1-Propanol"] == 1, 'alcohol'] = '1-Propanol'
dataset.loc[dataset["2-Butanol"] == 1, 'alcohol'] = '2-Butanol'
dataset.loc[dataset["2-propanol"] == 1, 'alcohol'] = '2-propanol'
dataset.loc[dataset["1-isobutanol"] == 1, 'alcohol'] = '1-isobutanol'"""

'dataset.loc[dataset["1-Octanol"] == 1, \'alcohol\'] = \'1-Octanol\'\ndataset.loc[dataset["1-Propanol"] == 1, \'alcohol\'] = \'1-Propanol\'\ndataset.loc[dataset["2-Butanol"] == 1, \'alcohol\'] = \'2-Butanol\'\ndataset.loc[dataset["2-propanol"] == 1, \'alcohol\'] = \'2-propanol\'\ndataset.loc[dataset["1-isobutanol"] == 1, \'alcohol\'] = \'1-isobutanol\''

In [6]:
# dodanie kolumny z wartościami liczbowymi dla każdego rodzaju alkoholu
dataset.loc[dataset["1-Octanol"] == 1, 'alcohol'] = 1
dataset.loc[dataset["1-Propanol"] == 1, 'alcohol'] = 2
dataset.loc[dataset["2-Butanol"] == 1, 'alcohol'] = 3
dataset.loc[dataset["2-propanol"] == 1, 'alcohol'] = 4
dataset.loc[dataset["1-isobutanol"] == 1, 'alcohol'] = 5
dataset['alcohol'].value_counts()

1.0    25
2.0    25
3.0    25
4.0    25
5.0    25
Name: alcohol, dtype: int64

In [7]:
print(dataset.head())

   0.799_0.201  0.799_0.201.1  0.700_0.300  0.700_0.300.1  0.600_0.400  \
0       -10.06         -10.62       -14.43         -18.31       -24.64   
1        -9.69         -10.86       -16.73         -21.75       -28.47   
2       -12.07         -14.28       -21.54         -27.92       -35.19   
3       -14.21         -17.41       -25.91         -33.36       -41.29   
4       -16.57         -20.35       -29.97         -37.84       -47.03   

   0.600_0.400.1  0.501_0.499  0.501_0.499.1  0.400_0.600  0.400_0.600.1  \
0         -30.56       -38.62         -45.59       -54.89         -62.28   
1         -35.83       -43.65         -52.43       -61.92         -71.27   
2         -43.94       -52.04         -62.49       -71.97         -83.10   
3         -51.27       -59.94         -71.55       -81.51         -93.83   
4         -57.29       -67.13         -78.96       -90.01        -102.65   

   1-Octanol  1-Propanol  2-Butanol  2-propanol  1-isobutanol  alcohol  
0          1           0 

In [8]:
# przygotowanie do klasyfikacji

In [9]:
"""
x = pd.DataFrame(dataset.values[:,:10], columns =[
    '0,799_0,201',
    '0,799_0,201.1',
    '700_0,300',
    '0,700_0,300.1',
    '0,600_0,400', 
    '0,600_0,400.1', 
    '0,501_0,499', 
    '0,501_0,499.1', 
    '0,400_0,600',
    '0,400_0,600.1'])
y = pd.DataFrame(dataset.values[:,15], columns = ['alcohol'])
"""

"\nx = pd.DataFrame(dataset.values[:,:10], columns =[\n    '0,799_0,201',\n    '0,799_0,201.1',\n    '700_0,300',\n    '0,700_0,300.1',\n    '0,600_0,400', \n    '0,600_0,400.1', \n    '0,501_0,499', \n    '0,501_0,499.1', \n    '0,400_0,600',\n    '0,400_0,600.1'])\ny = pd.DataFrame(dataset.values[:,15], columns = ['alcohol'])\n"

In [10]:
######################
# klasyfikator binarny
######################

In [11]:
x = np.array(dataset.values[:,:10])
y = np.array(dataset.values[:,15])

In [12]:
print(x[:5])
print(y[:5])

[[ -10.06  -10.62  -14.43  -18.31  -24.64  -30.56  -38.62  -45.59  -54.89
   -62.28]
 [  -9.69  -10.86  -16.73  -21.75  -28.47  -35.83  -43.65  -52.43  -61.92
   -71.27]
 [ -12.07  -14.28  -21.54  -27.92  -35.19  -43.94  -52.04  -62.49  -71.97
   -83.1 ]
 [ -14.21  -17.41  -25.91  -33.36  -41.29  -51.27  -59.94  -71.55  -81.51
   -93.83]
 [ -16.57  -20.35  -29.97  -37.84  -47.03  -57.29  -67.13  -78.96  -90.01
  -102.65]]
[1. 1. 1. 1. 1.]


In [13]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.9, random_state=42)

In [14]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(12, 10)
(12,)
(113, 10)
(113,)


In [15]:
# stratyfikacja
skf = StratifiedKFold(n_splits=5, shuffle=True)

In [16]:
# walidacja krzyżowa
# skalowanie maksymalnej iteracji
model = make_pipeline(StandardScaler(), LogisticRegression())
model.fit(x_train, y_train)
# pomiar dokładności bez stratyfikacji i walidacji krzyżowej
y_test_pred_from_model = model.predict(x_test)
print(accuracy_score(y_test, y_test_pred_from_model))
# pomiar dokładności ze stratyfikacją i walidacją krzyżową
print(cross_val_score(model, x, y, cv=skf))

0.4690265486725664
[0.64 0.52 0.48 0.64 0.6 ]


In [17]:
print(statistics.mean(cross_val_score(model, x, y, cv=skf)))

0.56


In [18]:
#################################
# klasyfikator drzewa decyzyjnego
#################################

In [19]:
x = np.array(dataset.values[:,:10])
y = np.array(dataset.values[:,15])

In [20]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.9, random_state=42)

In [21]:
print(x[:5])
print(y[:5])

[[ -10.06  -10.62  -14.43  -18.31  -24.64  -30.56  -38.62  -45.59  -54.89
   -62.28]
 [  -9.69  -10.86  -16.73  -21.75  -28.47  -35.83  -43.65  -52.43  -61.92
   -71.27]
 [ -12.07  -14.28  -21.54  -27.92  -35.19  -43.94  -52.04  -62.49  -71.97
   -83.1 ]
 [ -14.21  -17.41  -25.91  -33.36  -41.29  -51.27  -59.94  -71.55  -81.51
   -93.83]
 [ -16.57  -20.35  -29.97  -37.84  -47.03  -57.29  -67.13  -78.96  -90.01
  -102.65]]
[1. 1. 1. 1. 1.]


In [22]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(12, 10)
(12,)
(113, 10)
(113,)


In [23]:
# stratyfikacja plus walidacja krzyżowa
clf = tree.DecisionTreeClassifier()
skf = StratifiedKFold(n_splits=5, shuffle=True)
clf = clf.fit(x_train, y_train)
# dokładność
y_test_pred_from_clf = clf.predict(x_test)
print('bez walidacji krzyżowej i stratyfikacji', 
      accuracy_score(y_test, y_test_pred_from_clf))
print('walidacja i stratyfikacja',
      cross_val_score(clf, x, y, cv=skf))

bez walidacji krzyżowej i stratyfikacji 0.4778761061946903
walidacja i stratyfikacja [0.84 0.84 0.88 1.   0.96]


In [24]:
# optymalizacja modelu
criterion = np.array(['gini','entropy'])
max_depth = np.array([
    1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
grid = GridSearchCV(estimator = clf, 
                    param_grid = dict(
                        max_depth=max_depth, 
                        criterion=criterion))
grid.fit(x, y)
print(grid.best_params_)

{'criterion': 'entropy', 'max_depth': 13}


In [25]:
clf = tree.DecisionTreeClassifier(
    criterion = grid.best_params_['criterion'], 
    max_depth = grid.best_params_['max_depth'])
clf = clf.fit(x_train, y_train)
# dokładność
y_test_pred_from_clf = clf.predict(x_test)
print(accuracy_score(y_test, y_test_pred_from_clf))
print(cross_val_score(clf, x, y, cv=skf))

0.3893805309734513
[0.96 0.96 1.   0.96 0.84]


In [26]:
print(statistics.mean(cross_val_score(clf, x, y, cv=skf)))

0.896


In [27]:
############################
# klasyfikator lasu losowego
############################

In [50]:
x = np.array(dataset.values[:,:10])
y = np.array(dataset.values[:,15])

In [51]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.9, random_state=42)

In [52]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(12, 10)
(12,)
(113, 10)
(113,)


In [53]:
clf = RandomForestClassifier(n_estimators=2)
skf = StratifiedKFold(n_splits=5, shuffle=True)
clf = clf.fit(x_train, y_train)
# dokładność
y_test_pred_from_clf = clf.predict(x_test)
print(accuracy_score(y_test, y_test_pred_from_clf))
print(cross_val_score(clf, x, y, cv=skf))

0.34513274336283184
[0.8  0.84 0.72 0.76 0.96]


In [54]:
# optymalizacja modelu
criterion = np.array(['gini','entropy'])
max_depth = np.array(
    [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
n_estimators = np.array([2,3,4,5,6,7,8,9,10])
clf = RandomForestClassifier()
grid = GridSearchCV(estimator = clf, 
                    param_grid = dict(
                        n_estimators = n_estimators, 
                        max_depth=max_depth, 
                        criterion=criterion))
grid.fit(x, y)
print(grid.best_params_)

{'criterion': 'entropy', 'max_depth': 12, 'n_estimators': 5}


In [55]:
clf = RandomForestClassifier(
    n_estimators = grid.best_params_['n_estimators'], 
    criterion = grid.best_params_['criterion'], 
    max_depth = grid.best_params_['max_depth'])
clf = clf.fit(x_train, y_train)
# dokładność
y_test_pred_from_clf = clf.predict(x_test)
print(accuracy_score(y_test, y_test_pred_from_clf))
print(cross_val_score(clf, x, y, cv=skf))

0.5132743362831859
[1.   0.92 0.84 0.92 0.96]


In [56]:
print(statistics.mean(cross_val_score(clf, x, y, cv=skf)))

0.912


In [35]:
################################
# naiwny klasyfikator bayesowski
################################

In [36]:
x = np.array(dataset.values[:,:10])
y = np.array(dataset.values[:,15])

In [37]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.9, random_state=42)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(12, 10)
(12,)
(113, 10)
(113,)


In [38]:
clf = GaussianNB()
clf = clf.fit(x_train, y_train)

In [39]:
# dokładność
y_test_pred_from_clf = clf.predict(x_test)
print(accuracy_score(y_test, y_test_pred_from_clf))
skf = StratifiedKFold(n_splits=5, shuffle=True)
print(cross_val_score(clf, x, y, cv=skf))

0.2743362831858407
[0.4  0.6  0.48 0.44 0.4 ]


In [40]:
print(statistics.mean(cross_val_score(clf, x, y, cv=skf)))

0.424


In [41]:
################################
# klasyfikator sieci neuronowych
################################

In [42]:
x = np.array(dataset.values[:,:10])
y = np.array(dataset.values[:,15])

In [43]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.9, random_state=42)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(12, 10)
(12,)
(113, 10)
(113,)


In [44]:
clf = MLPClassifier(
    hidden_layer_sizes=(30), 
    random_state=42, 
    max_iter=5000)
clf = clf.fit(x_train, y_train)

In [45]:
# dokładność
y_test_pred_from_clf = clf.predict(x_test)
print(accuracy_score(y_test, y_test_pred_from_clf))
skf = StratifiedKFold(n_splits=5, shuffle=True)
print(cross_val_score(clf, x, y, cv=skf))

0.19469026548672566
[0.2  0.2  0.24 0.16 0.12]


In [46]:
print(statistics.mean(cross_val_score(clf, x, y, cv=skf)))

0.17600000000000002


In [47]:
# optymalizacja modelu
hidden_layer_sizes = np.array([5,10,30,50,100])
random_state = np.array([0,10,20,40,60])
max_iter = np.array([8000])
clf = MLPClassifier()
grid = GridSearchCV(estimator = clf, 
                    param_grid = dict(
                        hidden_layer_sizes=(hidden_layer_sizes),
                        random_state=random_state,
                        max_iter=max_iter))
grid.fit(x, y)
print(grid.best_params_)

{'hidden_layer_sizes': 30, 'max_iter': 8000, 'random_state': 20}


In [48]:
clf = MLPClassifier(
    hidden_layer_sizes = (grid.best_params_['hidden_layer_sizes']), 
    random_state = grid.best_params_['random_state'], 
    max_iter = grid.best_params_['max_iter'])
clf = clf.fit(x_train, y_train)
# dokładność
y_test_pred_from_clf = clf.predict(x_test)
print(accuracy_score(y_test, y_test_pred_from_clf))
print(cross_val_score(clf, x, y, cv=skf))

0.5398230088495575
[0.88 0.88 0.92 0.84 0.84]


In [49]:
print(statistics.mean(cross_val_score(clf, x, y, cv=skf)))

0.92
